In [11]:
import pyodbc
import sys
class Database:
    def __init__(self):
        pass

    def connect_db(self):
        try:
            self.conn = pyodbc.connect(
                r'Driver=ODBC Driver 18 for SQL Server;'
                r'Server=SEJAL\SQLEXPRESS;'
                r'Database=Courses_DB;'
                r'Trusted_Connection=yes;'
                r'TrustServerCertificate=yes;'
            )
            self.cursor = self.conn.cursor()
            print("Database connected successfully!")
        except Exception as e:
            print("Connection failed:", e)

    def show_student_courses(self):
        try:
            self.cursor.execute("SELECT * FROM dbo.StudentCourses")
            rows = self.cursor.fetchall()

            if rows:
                print("\n----- Student Courses Data -----")
                for row in rows:
                    print(row)
            else:
                print("No records found.")

        except Exception as e:
            print("Error fetching data:", e)
            
class Student:
    def __init__(self, db):
        self.db = db
        self.username = None

    def signup(self):
        username = input("Enter username: ")
        password = input("Enter password: ")
        try:
            self.db.cursor.execute(
                "INSERT INTO dbo.Students (Username, Password) VALUES (?, ?)",
                (username, password)
            )
            self.db.conn.commit()
            print("Signup successful! Please login now.")
        except Exception as e:
            print("Signup failed:", e)

    def login(self):
        username = input("Enter username: ")
        password = input("Enter password: ")
        self.db.cursor.execute(
            "SELECT * FROM dbo.Students WHERE Username=? AND Password=?",
            (username, password)
        )
        row = self.db.cursor.fetchone()
        if row:
            print(f"Login successful! Welcome {username}")
            self.username = username
            return True
        else:
            print("Invalid username or password.")
            return False

class OptCoursesDB:
    def __init__(self, db, student_username):
          
        self.db = db
        self.student_username = student_username
        self.courses = ["HR", "Finance", "Marketing", "DS"]
        self.list_sem = ['semester', "annual", "no"]
        self.base_fees = 200000
        self.analytics_fee = self.base_fees * 0.10
        self.hostel_rent = 200000
        self.transportation_charges = 13000
        self.food_per_month = 2000
        self.total_cost = 0

       
        self.conn = None
        self.cursor = None

    def connect_db(self):
        try:
            self.conn = pyodbc.connect(
                r'Driver=ODBC Driver 18 for SQL Server;'
                r'Server=SEJAL\SQLEXPRESS;'
                r'Database=Courses_DB;'
                r'Trusted_Connection=yes;'
                r'TrustServerCertificate=yes;'
            )
            self.cursor = self.conn.cursor()
            print("Database connected successfully!")
        except Exception as e:
            print("Connection failed:", e)
            sys.exit()

    def select_course(self):
        print(f"Available courses for Master's degree: {self.courses}")
        while True:
            subject = input("Enter Subject: ")
            if subject in self.courses:
                print(f"You have selected {subject}")
                self.analytics = input("Have you applied for analytics? (Y/N): ").lower()
                self.total_cost = self.base_fees

                if self.analytics == 'y':
                    print("You have opted for analytics")
                    self.total_cost += self.analytics_fee

                self.extra_facilities()
                self.store_to_db(subject)
                break
            else:
                print("Please select a valid course")

    

    def extra_facilities(self):
        
        self.hostel = input("Do you need hostel? (Y/N): ").lower()
        if self.hostel == 'y':
            self.total_cost += self.hostel_rent

        
        self.food_months = int(input("For how many months do you need food facility? "))
        if self.food_months > 0:
            self.total_cost += self.food_months * self.food_per_month

       
        self.transportation = input("Do you need transportation? (semester/annual/no): ").lower()
        if self.transportation in self.list_sem and self.transportation != 'no':
            self.total_cost += self.transportation_charges

        print("Your total fee is:", self.total_cost)

    def store_to_db(self, subject):
        try:
            self.db.cursor.execute("""
                INSERT INTO dbo.StudentCourses 
                (Username, Course, Analytics, Hostel, FoodMonths, Transportation, TotalFee)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (
                self.student_username,
                subject,
                self.analytics,
                self.hostel,
                self.food_months,
                self.transportation,
                self.total_cost
            ))

            self.db.conn.commit()
            print("Data stored successfully!")

        except Exception as e:
            print("Insert failed:", e)
db = Database()
db.connect_db()

student_obj = Student(db)
choice = input("Do you have an account? (Y/N): ").lower()

if choice == 'n':
    student_obj.signup()

if student_obj.login():
    course = OptCoursesDB(db, student_obj.username)
    course.select_course()
    db.show_student_courses()


Database connected successfully!
Login successful! Welcome sejal kumbhar
Available courses for Master's degree: ['HR', 'Finance', 'Marketing', 'DS']
You have selected HR
You have opted for analytics
Your total fee is: 449000.0
Data stored successfully!

----- Student Courses Data -----
(1, 'HR', 'y', 'y', 3, 'y', 426000.0, None)
(2, 'HR', 'y', 'y', 5, 'annual', 443000.0, 'sejal kumbhar')
(3, 'Marketing', 'y', 'y', 3, 'no', 426000.0, 'sejal kumbhar')
(4, 'HR', 'y', 'y', 8, 'annual', 449000.0, 'sejal kumbhar')
